In [ ]:
# Useful starting lines
%matplotlib inline

import numpy as np
import scipy
import scipy.io
import scipy.sparse as sp
import matplotlib.pyplot as plt
%load_ext autoreload
%autoreload 2

## Load the Data
Note that `ratings` is a sparse matrix that in the shape of (num_items, num_users)

In [ ]:
from helpers import load_data, preprocess_data

path_dataset = "data/data_train.csv"
ratings = load_data(path_dataset)

### Plot the number of ratings per movie and user

In [ ]:
from plots import plot_raw_data

num_items_per_user, num_users_per_item = plot_raw_data(ratings)

print("min # of items per user = {}, min # of users per item = {}.".format(
        min(num_items_per_user), min(num_users_per_item)))

### Split the data into a train and test set

In [ ]:
from plots import plot_train_test_data
from helpers import split_data

valid_ratings, train_unfiltered, test = split_data(
    ratings, num_items_per_user, num_users_per_item, min_num_ratings=10, p_test=0.1)


In [ ]:
from outliers_filtering import *
#d = disagreements(train_unfiltered)
#plot_disagreements(d)
#train = filter_outliers(train_unfiltered,d)
#nfiltered = train_unfiltered.nnz - train.nnz
#print("number of filtered ratings : {}".format(nfiltered))
train = train_unfiltered

In [ ]:
from outliers_filtering import threshold_tests
threshold_tests(d, train_unfiltered, valid_ratings, test)

## Implementing Baselines 

### Use the global mean to do the prediction

In [ ]:
from helpers import calculate_mse
from baselines import baseline_global_mean, compute_rmse

train_mean, test_mean = baseline_global_mean(train, test)
pred = np.ones(train.shape) * train_mean
test_pred = np.ones(test.shape) * test_mean


rmse_test = compute_rmse(test, test_pred)
rmse_submission = compute_rmse(valid_ratings, pred)
print(rmse_test)
print(rmse_submission)

### Use the user means as the prediction

In [ ]:
from baselines import baseline_user_mean

train_means, test_means = baseline_user_mean(train, test)
train_means_list = train_means.tolist()
pred = np.ones(train.shape)
for col in range(train.shape[1]):
    pred[:,col] *= train_means_list[0][col]
test_means_list = test_means.tolist()
test_pred = np.ones(test.shape)
for col in range(test.shape[1]):
    test_pred[:,col] *= test_means_list[0][col]

rmse_test = compute_rmse(test, test_pred)
rmse_submission = compute_rmse(valid_ratings, pred)
print(rmse_test)
print(rmse_submission)

### Use the item means as the prediction

In [ ]:
from baselines import baseline_item_mean
from helpers import exportSubmission

train_means, test_means = baseline_item_mean(train, test)
train_means_list = train_means.tolist()
pred = np.ones(train.shape)
for col in range(train.shape[0]):
    pred[col,:] *= train_means_list[col]
test_means_list = test_means.tolist()
test_pred = np.ones(test.shape)
for col in range(test.shape[0]):
    test_pred[col,:] *= test_means_list[col]

rmse_test = compute_rmse(test, test_pred)
rmse_submission = compute_rmse(valid_ratings, pred)
print(rmse_test)
print(rmse_submission)

exportSubmission("data/submission_item_mean.csv", pred)

### Learn the Matrix Factorization using SGD

#### Initialize matrix factorization

Compute the cost by the method of matrix factorization.


In [ ]:
from matrix_factorization import matrix_factorization_SGD
user_features, item_features = matrix_factorization_SGD(train, test)

In [ ]:
from helpers import compute_error
print(compute_error(valid_ratings, user_features, item_features, valid_ratings.nonzero()))

In [ ]:
pred =  (item_features @ user_features.T)
exportSubmission("data/submission_MF.csv", pred)

In [ ]:
errors = []
for i in np.arange(10, 20, 2):
    user_features, item_features = matrix_factorization_SGD(train, test, num_epochs = 15, num_features = i)
    error = compute_error(valid_ratings, user_features, item_features, valid_ratings.nonzero())
    errors.append(error)


In [ ]:
from plots import visualization
visualization(np.arange(10, 20, 2),errors,errors)

### Learn the Matrix Factorization using Alternating Least Squares

In [ ]:
def update_user_feature(
        train, item_features, lambda_user,
        nnz_items_per_user, nz_user_itemindices):
    """update user feature matrix."""
    # ***************************************************
    # INSERT YOUR CODE HERE
    # TODO
    # update and return user feature.
    # ***************************************************
    raise NotImplementedError

def update_item_feature(
        train, user_features, lambda_item,
        nnz_users_per_item, nz_item_userindices):
    """update item feature matrix."""
    # ***************************************************
    # INSERT YOUR CODE HERE
    # TODO
    # update and return item feature.
    # ***************************************************
    raise NotImplementedError

In [ ]:
from helpers import build_index_groups


def ALS(train, test):
    """Alternating Least Squares (ALS) algorithm."""
    # define parameters
    num_features = 20   # K in the lecture notes
    lambda_user = 0.1
    lambda_item = 0.7
    stop_criterion = 1e-4
    change = 1
    error_list = [0, 0]
    
    # set seed
    np.random.seed(988)

    # init ALS
    user_features, item_features = init_MF(train, num_features)
    
    # ***************************************************
    # INSERT YOUR CODE HERE
    # TODO
    # start you ALS-WR algorithm.
    # ***************************************************
    raise NotImplementedError

ALS(train, test)